# Robust Lot Sizing

The Lot Sizing Problem (LSP) is a fundamental optimization challenge in operations management, that focuses on finding the best quantities to produce or order in each time period to meet demand while minimizing costs. Over a fixed planning horizon, the problem involves known product demand and associated costs like setup, production, inventory holding, and backorders. The primary goal is to devise a production plan that efficiently meets demand each time period. In its robust form, the problem considers uncertain production yields, fluctuating within a range defined by its nominal value and maximum deviation.

In this example, we aim to solve the Robust LSP provided Metzker et al. (2023) [1] in Section 3.3. Let us consider a single item multi period uncapacitated LSP with backorder and production yield that determine the quantity to produce in each time period of the finite planning horizon $T$. We are given the setup cost $s$, the inventory holding cost $h$, the backorder cost $b$ and the demand $d$ for each time period in $T$. The model contains the decision variables $X$, the lot size to be produced, and the setup decision $Y$. The strictly positive uncertain production yield is defined as $\bar{p}$. $H$ is the total holding and backorder cost per period.

The robust problem can be optimization as follows:

$$
\begin{array}{ll}
\text{minimize} & s^TY + v^TX + H \\
\text{subject to} & H_t \geq h_t[ \bar{p}^TX_t - d] \\
                  & H_t \geq -b_t[ \bar{p}^TX_t - d] \\
                  & X_t \leq M_tY_t \\
                  &  X_t \geq 0\\
                  & H_t \geq 0

\end{array}
$$

To solve this problem, we first import the required packages.

In [1]:
import numpy as np
import lropt
import cvxpy as cp

We start by defining the relevant constants and creating variables. 

In [2]:
np.random.seed(1)
T = 5 
s = np.random.rand(T)  
v = np.random.rand(T)  
h = np.random.rand(T)  
b = np.random.rand(T)  
d = np.random.randint(10, 20, size=T)  
RHO = 2
# Large positive number for setup constraint
M = np.full(T, 1000)


X = cp.Variable(T, nonneg=True)
Y = cp.Variable(T, boolean=True)  
H = cp.Variable(T, nonneg=True)  
NUM_DEC = 3 


In the following we define the parameter $p$ as an uncertain parameter that belongs a Budget Uncertainty set. The budget uncertainty is defined as:
$$
        \mathcal{U}_{\text{budget}} = \{z \ | \ \|z \|_\infty \le \rho_1,
        \|z \|_1 \leq \rho_2\}
$$

In [3]:
p = lropt.UncertainParameter(T, uncertainty_set = lropt.Budget(rho1 = RHO, rho2= RHO), nonneg = True)

We define our objective and constraints. We solve the problem and get the optimal value. 

In [ ]:
objective = cp.Minimize(cp.sum(s @ Y + v @ X + H))
constraints = [
H >= h * cp.sum(p @ X - d),
H >= (-b) * cp.sum(p @ X - d),
X <= M@Y
]

In [5]:
prob = lropt.RobustProblem(objective, constraints)
a = prob.solve()

/Users/mj5676/Desktop/miniconda3/envs/lropt_v3/lib/python3.12/site-packages/scipy/sparse/_index.py:134: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)
/Users/mj5676/Desktop/miniconda3/envs/lropt_v3/lib/python3.12/site-packages/cvxpy/utilities/torch_utils.py:61: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:643.)
  return torch.sparse.FloatTensor(i, v, torch.Size(value_coo.shape)).to(dtype)


In [6]:
print(f"The robust optimal values for the total cost using Budget uncertainty are {np.array(H.value).round(3)}")

The robust optimal values for the total cost using Budget uncertainty are [60.342 37.557 50.282 12.635 17.829]


## References

1. Metzker, P., Thevenin, S., Adulyasak, Y., & Dolgui, A. (2023). Robust optimization for lot-sizing problems under yield uncertainty. https://doi.org/10.1016/j.cor.2022.106025